# Praktikum 0
## Mathematical modeling of the atmosphere


### Integrating simple closure equations for moisture

Consider the following set of 1D equations for moisture variables. The domain is $z=[0,1]$
First, the grid-mean total water mixing ratio, $\overline{r_t}(t,z)$, depends on the vertical flux of $r_t$, $\overline{w'r'_t}(t,z)$:

$$\frac{\delta \overline{r_t}}{\delta t} = -\frac{\delta}{\delta z}\overline{w'r'_t}$$
Second, the flux $\overline{w'r'_t}$ is assumed to be related to $\overline{r_t}$ by

$$\overline{w'r'_t}=-K\frac{\delta\overline{r_t}}{\delta z}$$

That is, the flux is determined by down-gradient diffusion. The boundary condition at the
boundaries $z=[0,1]$ is $\overline{w'r'_t}=0$.
Finally, the sub-grid variance of total water, $\overline{{r'_t}^2}(t,z)$, is assumed to be adequately diagnosed by the algebraic expression

$$\overline{{r'_t}^2}=\tau K (\frac{\delta \overline{r_t}}{\delta z})^2$$

where $\tau=1$ is a prescribed eddy turnover time, and $K=K(z)$ is a prescribed eddy diffusivity:

$$K(z) = \begin{cases} 1, & \text{   if $z<=1/2$} \\ z + 1/2, & \text{  $1/2<z<=1$}\end{cases}$$
The initial profile of total water diminishes with height above the mixed layer:

$$\overline{r_t}(t=0,z) = \begin{cases} r_{t,sfc}, & \text{   if $z<=1/2$} \\ -r_{t,sfc}(z-1/2) + r_{t,sfc}, & \text{  $1/2<z<=1$} \end{cases}$$

where $r_{t,sfc}=10$.

In [176]:
"""Assuming t_t0(z) for z outside the boarders equals 1."""

rt_srf = 10


def r_t0(z):
    if z > 1:
        return 1
    elif z > 0.5:
        return -rt_srf * (z - 0.5) + rt_srf
    else:
        return rt_srf

In [177]:
"""Assuming k(z) for z outside the boarders equals 1."""


def k(z):
    if z > 1:
        return 1
    elif z > 0.5:
        return z + 0.5
    else:
        return 1

So equation (1) in (2) is now:

$$\frac{\delta \overline{r_t}}{\delta t}=\frac{\delta}{\delta z}(K\cdot\frac{\delta \overline{r_t}}{\delta z})$$

We will have a vector $\overline{r_t}$ which represents the searched values.

In [178]:
import numpy as np

n = 100
# Create vector with n values which are equally spaced between 0 and 10.
r_t = np.linspace(0, 1, n)

# Create a height vector with the same dimension, too.
height = np.copy(r_t)

# Set up the initial condition.
for i in range(len(r_t)):
    r_t[i] = r_t0(height[i])

print(r_t)
print(np.average(r_t))

[10.         10.         10.         10.         10.         10.
 10.         10.         10.         10.         10.         10.
 10.         10.         10.         10.         10.         10.
 10.         10.         10.         10.         10.         10.
 10.         10.         10.         10.         10.         10.
 10.         10.         10.         10.         10.         10.
 10.         10.         10.         10.         10.         10.
 10.         10.         10.         10.         10.         10.
 10.         10.          9.94949495  9.84848485  9.74747475  9.64646465
  9.54545455  9.44444444  9.34343434  9.24242424  9.14141414  9.04040404
  8.93939394  8.83838384  8.73737374  8.63636364  8.53535354  8.43434343
  8.33333333  8.23232323  8.13131313  8.03030303  7.92929293  7.82828283
  7.72727273  7.62626263  7.52525253  7.42424242  7.32323232  7.22222222
  7.12121212  7.02020202  6.91919192  6.81818182  6.71717172  6.61616162
  6.51515152  6.41414141  6.31313131  6.21

Exercises:

- Integrate the equations forward in time using a discretization method of your choice.
- Plot the profiles of $\overline{r_t}$ and $\overline{{r'_t}^2}$ for later times.
    Can you explain their behavior in physical terms?
    What do you expect the behavior will be as $t\to\infty$?

So we are assuming now that:

$$\frac{\delta \overline{r_t}}{\delta t} \approx \frac{\overline{{r_i}^{n+1}} - \overline{r_i^n}}{\Delta t}$$
And:

$$\frac{\delta^2 \overline{r_t}}{\delta z^2} \approx \frac{1}{\Delta z^2}(\overline{{r_{i+1}^n}} - 2\cdot\overline{r_i^n} + \overline{{r_{i-1}^n}})$$

So in conclusion

$$\frac{\delta \overline{r_t}}{\delta t}=\frac{\delta}{\delta z}(K\cdot\frac{\delta \overline{r_t}}{\delta z})$$
### First Exercise:
becomes

$$\frac{\overline{{r_i}^{n+1}} - \overline{r_i^n}}{\Delta t} = \frac{K}{\Delta z^2}(\overline{{r_{i+1}^n}} - 2\cdot\overline{r_i^n} + \overline{{r_{i-1}^n}})$$
$$\Rightarrow \overline{r_i^{n+1}} = \frac{K\cdot\Delta t}{\Delta z^2}(\overline{{r_{i+1}^n}} - 2\cdot\overline{r_i^n} + \overline{{r_{i-1}^n}}) + \overline{r_i^n}$$
With K dependent on z so actually the index values.

$$\Rightarrow \overline{r_i^{n+1}} = \frac{K(i)\cdot\Delta t}{\Delta z^2}(\overline{{r_{i+1}^n}} - 2\cdot\overline{{r_i}^n} + \overline{{r_{i-1}^n}}) + \overline{r_i^n}$$
Equivalent:

$$\Rightarrow \overline{r_i^{n+1}} = (\frac{K(i)\cdot\Delta t}{\Delta z^2}) \cdot \overline{r_{i+1}^n} - (1 - 2\cdot (\frac{K(i)\cdot\Delta t}{\Delta z^2})) \cdot \overline{r_i^n} + (\frac{K(i)\cdot\Delta t}{\Delta z^2}) \cdot \overline{r_{i-1}^n}$$
Normally we would now create a matrix A to solve

$$A \cdot \overline{r_i^n} = \overline{r_i^{n+1}}$$
With $- (1 - 2\cdot (\frac{K(i)\cdot\Delta t}{\Delta z^2}))$ at the diagonal, $(\frac{K(i)\cdot\Delta t}{\Delta z^2})$ under the diagonal and $(\frac{K(i)\cdot\Delta t}{\Delta z^2})$ over the diagonal, and all other entries $0$.
Because $K$ depends on $z$ we unfortunately cannot do the matrix vector multiplication because we cannot get the right index. So I am going to do it with loops, a very slow variant.

In [179]:
def forward_euler(r, z, delta_t, delta_z):
    # Create a new target vector r_t1
    r_t1 = np.zeros(n)

    for index in range(len(r)):
        if index - 1 >= 0 and index + 1 < len(r):
            a = (k(z[index]) * delta_t) / delta_z ** 2
            r_t1[index] = a * (r[index + 1] - 2 * r[index] + r[index - 1]) + r[index]
        r_t1[0] = r_t1[1]
        r_t1[len(r) - 1] = r_t1[len(r) - 2]

    return r_t1

In [180]:
for i in range(50000):
    if i % 10000 == 0:
        print(r_t)
        print('Average:', np.average(r_t))
    r_t = forward_euler(r_t, height, 1e-6, 1 / n)

[10.         10.         10.         10.         10.         10.
 10.         10.         10.         10.         10.         10.
 10.         10.         10.         10.         10.         10.
 10.         10.         10.         10.         10.         10.
 10.         10.         10.         10.         10.         10.
 10.         10.         10.         10.         10.         10.
 10.         10.         10.         10.         10.         10.
 10.         10.         10.         10.         10.         10.
 10.         10.          9.94949495  9.84848485  9.74747475  9.64646465
  9.54545455  9.44444444  9.34343434  9.24242424  9.14141414  9.04040404
  8.93939394  8.83838384  8.73737374  8.63636364  8.53535354  8.43434343
  8.33333333  8.23232323  8.13131313  8.03030303  7.92929293  7.82828283
  7.72727273  7.62626263  7.52525253  7.42424242  7.32323232  7.22222222
  7.12121212  7.02020202  6.91919192  6.81818182  6.71717172  6.61616162
  6.51515152  6.41414141  6.31313131  6.21

In [181]:
# import matplotlib.pyplot as plt
#
# fig, axs = plt.subplots(nrows=5, ncols=2, figsize=(10, 10), constrained_layout=True)
# for i in range(10):
#     ax = axs[i // 2, i % 2]
#     ax.plot(height, r_t)
#     ax.set(xlabel='z', ylabel='r_t', title=f"t={i}")
#     r_t = np.copy(forward_euler(r_t, height))
#
# plt.show()